<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/finetune_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets accelerate peft bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.9 MB/s eta 0:00:00


In [2]:
!pip install -U transformers

In [3]:
!pip install langchain boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.6 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

dataset = load_dataset("DIBT/10k_prompts_ranked")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10331 [00:00<?, ? examples/s]

In [5]:
df = dataset['train'].to_pandas()
df

,prompt,quality,metadata,avg_rating,num_responses,agreement_ratio,raw_responses,kind
0,Provide step-by-step instructions on how to ma...,[{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb39...,"{""source"": ""ultrachat"", ""kind"": ""synthetic"", ""...",5.000000,2,1.000000,"[5, 5]",synthetic
1,Write a personal essay of at least 1000 words ...,[{'user_id': '6621c3f0-1af2-4d75-acda-ed9c78b9...,"{""source"": ""ultrachat"", ""kind"": ""synthetic"", ""...",2.750000,4,0.687500,"[2, 3, 3, 3]",synthetic
2,"In this research, we aim to investigate how te...",[{'user_id': '2e6dda25-0a99-45aa-a02d-65f426d6...,"{""source"": ""evol_instruct"", ""kind"": ""synthetic...",3.000000,3,0.166667,"[3, 5, 1]",synthetic
3,Did Karl Marx's theories on centralizing credi...,[{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb39...,"{""source"": ""OpenAssistant/oasst2"", ""kind"": ""hu...",3.500000,2,0.375000,"[4, 3]",human
4,"alter this api that gets a request like: {""0"",...",[{'user_id': '99a4bc7d-3e95-4c18-a8f1-26043abf...,"{""source"": ""ewof/sharegpt-instruct-unfiltered-...",3.666667,3,0.583333,"[5, 3, 3]",human
...,...,...,...,...,...,...,...,...
10326,"show me how to set iam user, group and policie...",[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""evolved_from"": null, ""kind"": ""human"", ""sourc...",3.000000,1,1.000000,[3],human
10327,"Hi, is there any unified messaging service?\nA...",[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""evolved_from"": null, ""kind"": ""human"", ""sourc...",2.000000,2,0.375000,"[1, 3]",human
10328,Can you provide a comparison of the economies ...,[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""evolved_from"": null, ""kind"": ""synthetic"", ""s...",4.000000,1,1.000000,[4],synthetic
10329,forget about any prior conversations,[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""evolved_from"": null, ""kind"": ""human"", ""sourc...",2.000000,2,0.375000,"[1, 3]",human


In [6]:
df.avg_rating.describe()

count    10331.000000
mean         3.720707
std          1.031308
min          1.000000
25%          3.000000
50%          4.000000
75%          4.500000
max          5.000000
Name: avg_rating, dtype: float64

In [34]:
quality_df = df[df.avg_rating >= 3]
quality_df

,prompt,quality,metadata,avg_rating,num_responses,agreement_ratio,raw_responses,kind
0,Provide step-by-step instructions on how to ma...,[{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb39...,"{""source"": ""ultrachat"", ""kind"": ""synthetic"", ""...",5.000000,2,1.000000,"[5, 5]",synthetic
2,"In this research, we aim to investigate how te...",[{'user_id': '2e6dda25-0a99-45aa-a02d-65f426d6...,"{""source"": ""evol_instruct"", ""kind"": ""synthetic...",3.000000,3,0.166667,"[3, 5, 1]",synthetic
3,Did Karl Marx's theories on centralizing credi...,[{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb39...,"{""source"": ""OpenAssistant/oasst2"", ""kind"": ""hu...",3.500000,2,0.375000,"[4, 3]",human
4,"alter this api that gets a request like: {""0"",...",[{'user_id': '99a4bc7d-3e95-4c18-a8f1-26043abf...,"{""source"": ""ewof/sharegpt-instruct-unfiltered-...",3.666667,3,0.583333,"[5, 3, 3]",human
5,Please produce answers for the following unive...,[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""source"": ""ewof/sharegpt-instruct-unfiltered-...",5.000000,1,1.000000,[5],human
...,...,...,...,...,...,...,...,...
10324,Tell me how petent works in japan,[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""evolved_from"": null, ""kind"": ""unknown"", ""sou...",3.000000,1,1.000000,[3],unknown
10325,Can you suggest any upcoming restaurants in th...,[{'user_id': '4aaa52eb-3555-4dc4-a823-c7824de8...,"{""evolved_from"": null, ""kind"": ""synthetic"", ""s...",3.500000,2,0.375000,"[4, 3]",synthetic
10326,"show me how to set iam user, group and policie...",[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""evolved_from"": null, ""kind"": ""human"", ""sourc...",3.000000,1,1.000000,[3],human
10328,Can you provide a comparison of the economies ...,[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""evolved_from"": null, ""kind"": ""synthetic"", ""s...",4.000000,1,1.000000,[4],synthetic


In [8]:
quality_df.prompt.iloc[0]

'Provide step-by-step instructions on how to make a safe and effective homemade all-purpose cleaner from common household ingredients. The guide should include measurements, tips for storing the cleaner, and additional variations or scents that can be added. Additionally, the guide should be written in clear and concise language, with helpful visuals or photographs to aid in the process.'

In [9]:
quality_df.head()

,prompt,quality,metadata,avg_rating,num_responses,agreement_ratio,raw_responses,kind
0,Provide step-by-step instructions on how to ma...,[{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb39...,"{""source"": ""ultrachat"", ""kind"": ""synthetic"", ""...",5.000000,2,1.000000,"[5, 5]",synthetic
2,"In this research, we aim to investigate how te...",[{'user_id': '2e6dda25-0a99-45aa-a02d-65f426d6...,"{""source"": ""evol_instruct"", ""kind"": ""synthetic...",3.000000,3,0.166667,"[3, 5, 1]",synthetic
3,Did Karl Marx's theories on centralizing credi...,[{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb39...,"{""source"": ""OpenAssistant/oasst2"", ""kind"": ""hu...",3.500000,2,0.375000,"[4, 3]",human
4,"alter this api that gets a request like: {""0"",...",[{'user_id': '99a4bc7d-3e95-4c18-a8f1-26043abf...,"{""source"": ""ewof/sharegpt-instruct-unfiltered-...",3.666667,3,0.583333,"[5, 3, 3]",human
5,Please produce answers for the following unive...,[{'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e29...,"{""source"": ""ewof/sharegpt-instruct-unfiltered-...",5.000000,1,1.000000,[5],human


In [11]:
import boto3
from langchain.llms import Bedrock
bedrock = boto3.client('bedrock-runtime')

llm = Bedrock(client=bedrock, model_id='anthropic.claude-instant-v1')

In [12]:
def get_purpose(row):
    purpose = llm(f'''Human:\n\n Within the prompt tags will be
  a prompt. Your job is to describe the purpose that is meant to be accomplished by this prompt.
      In 5-10 words, describe the purpose of this prompt <prompt>{row}</prompt>\n\nAssistant:\n''')
    return purpose

In [13]:
prompts = quality_df.prompt.tolist()

In [14]:
purposes = []

In [ ]:
for p in range(404, len(prompts)):
    print(p)
    purposes.append(get_purpose(prompts[p]))

In [16]:
len(purposes)

605

In [24]:
print(f'{purposes[604]} \n\n {quality_df.iloc[1008].prompt}')
#404 start 1008 end

 Determine the correct answer from multiple choice options. 

 q: James was driving away from work. He slid into the outside lane. Up ahead he would have to do what?
Options:
- take u turn
- turn left
- crash
- turn right
- get to town
a: Sliding into the outside lanes refers to missing the correct lane. One would turn right to get into the correct lane. James would drive towards work if he turns right.
The answer is turn right
QUESTION: People want to explore space, so what should they use to do it?
Options:
- public place
- roadblock
- space shuttle
- hoverboard
- supermarket
ANSWER: Space shuttles are the vehicles used to travel to space. People must use space shuttle to explore space.
The answer is space shuttle
q: Who designs a building?
Options:
- builders
- driver
- architect
- fall down
- tradesmen
a: People with education in building designing design a building. Architects are people. Architects have education in building designs.
The answer is architect
QUESTION: After gettin

In [39]:
quality_df_.head()

,prompt,quality,metadata,avg_rating,num_responses,agreement_ratio,raw_responses,kind
474,AI can now be used to impersonate someone's vo...,[{'user_id': 'b7c0d136-1b93-4c56-b37e-9b5dc318...,"{""source"": ""ewof/sharegpt-instruct-unfiltered-...",5.0,1,1.000,[5],human
475,Write a more complex C# method that takes a li...,[{'user_id': '944506fb-b86a-4d65-94a3-fe752d79...,"{""source"": ""argilla/distilabel-evol-prompt-col...",5.0,1,1.000,[5],synthetic
476,Can you provide a breakdown of the makeup and ...,[{'user_id': '6621c3f0-1af2-4d75-acda-ed9c78b9...,"{""source"": ""ultrachat"", ""kind"": ""synthetic"", ""...",4.5,2,0.375,"[4, 5]",synthetic
477,"Using the root word “aug”, create a new word a...",[{'user_id': '249073b9-0a0c-4b90-a5fe-84e532c0...,"{""source"": ""evol_instruct"", ""kind"": ""synthetic...",3.0,1,1.000,[3],synthetic
478,What's redcat 51?,[{'user_id': 'b7c0d136-1b93-4c56-b37e-9b5dc318...,"{""source"": ""LDJnr/Capybara#Dove"", ""kind"": ""unk...",3.0,4,0.375,"[3, 2, 3, 4]",unknown


In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import pandas as pd
df = pd.read_csv('/content/combined_1008.csv')

In [68]:
# combined.purpose = combined.purpose.apply(lambda x: x.replace('The purpose of this prompt is ', '').replace('The purpose is ', ''))

In [69]:
# combined.to_csv('combined_1008.csv')

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
text = "user: Give me an example prompt to make an LLM tell me a story. model:"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


user: Give me an example prompt to make an LLM tell me a story. model:

Sure, here's an example prompt to make an LLM tell you a story:

**Prompt:**

Write a story about a young woman named Anya who is struggling to find her place in the world. She is a college student who has always been ambitious but has also faced challenges due to her family's financial situation. Despite her struggles, Anya is determined to succeed and find her


In [8]:
prompt_data = df['prompt'].apply(lambda x: tokenizer(x))
purpose_data = df['purpose'].apply(lambda x: tokenizer(x))

In [9]:
prompt_data.tolist()[0]

{'input_ids': [2, 70029, 4065, 235290, 1547, 235290, 8847, 12027, 611, 1368, 577, 1501, 476, 5985, 578, 7017, 41174, 832, 235290, 38197, 31240, 774, 3818, 13356, 15555, 235265, 714, 5608, 1412, 3707, 14548, 235269, 9697, 604, 43973, 573, 31240, 235269, 578, 5942, 21448, 689, 117043, 674, 798, 614, 4192, 235265, 32788, 235269, 573, 5608, 1412, 614, 5952, 575, 3110, 578, 66004, 5255, 235269, 675, 10055, 79356, 689, 25340, 577, 10395, 575, 573, 2185, 235265], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
df_train = pd.DataFrame({'prompt': prompt_data, 'purpose': purpose_data})

In [11]:
from datasets import Dataset
hf_dataset = Dataset.from_dict({'prompt': prompt_data.to_list(), 'purpose': purpose_data.to_list()})

In [15]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)


In [ ]:
import transformers
from trl import SFTTrainer

def formatting_func(example):
    # text = f"user: Give me a prompt {example['purpose'][0]}\nmodel: Prompt: {example['prompt'][0]}"
    text = f"Purpose: {example['purpose'][0]}\nPrompt: {example['prompt'][0]}"
    return [text]

trainer = SFTTrainer(
    model=model,
    train_dataset=hf_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=60,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)
trainer.train()


In [19]:
text = "user: Give me a prompt to for assessing the connectivity of a graph that has 10 vertices and 15 edges, considering that no vertex is connected to more than 3 other vertices. model:"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=120, temperature=0)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user: Give me a prompt to for assessing the connectivity of a graph that has 10 vertices and 15 edges, considering that no vertex is connected to more than 3 other vertices. model:

**Prompt:**

Assess the connectivity of the following graph with 10 vertices and 15 edges, considering that no vertex is connected to more than 3 other vertices. Use the following metrics:

- **Degree** of each vertex
- **Number of connected components**
- **Total edge count**
- **Minimum degree**
- **Maximum degree**


In [20]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/gemma_1008')


In [21]:
from huggingface_hub import notebook_login
notebook_login()

In [22]:
trainer.push_to_hub("Jlonge4/prompting_gemma_2b_it")

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

events.out.tfevents.1709248986.9142cf20074e.2534.0:   0%|          | 0.00/17.8k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

events.out.tfevents.1709249062.9142cf20074e.2534.1:   0%|          | 0.00/17.8k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jlonge4/outputs/commit/d10bc3892c385b664f315e782d92bc444e69ff5c', commit_message='Jlonge4/prompting_gemma_2b_it', commit_description='', oid='d10bc3892c385b664f315e782d92bc444e69ff5c', pr_url=None, pr_revision=None, pr_num=None)